## Setup

(1) Install IPyKernel

```console
pip install ipykernel
```

(2) Install FlexMeasures IPython Kernel

```console
ipython -m ipykernel install --user --name=fm
```

In [ ]:
from flexmeasures.app import create
from flexmeasures.data.models.time_series import Sensor

from datetime import timedelta

from flexmeasures.data.utils import save_to_db

from sqlalchemy import select, update
import time

import timely_beliefs as tb
import pandas as pd
import numpy as np

from flexmeasures.data.services.utils import get_or_create_model
from flexmeasures import Asset, Sensor, AssetType, Account, Source
from flexmeasures.data.models.time_series import TimedBelief

app = create()

In [ ]:
ctx = app.app_context()

In [ ]:
ctx.__enter__()

In [ ]:
from flexmeasures.data.models.planning.linear_optimization import device_scheduler
import pandas as pd

COLUMNS = [
    "equals",
    "max",
    "min",
    "efficiency",
    "derivative equals",
    "derivative max",
    "derivative min",
    "derivative down efficiency",
    "derivative up efficiency",
    "stock delta",
]

In [ ]:
import numpy as np


resolution = timedelta(minutes=15)
dt = pd.date_range("2024-01-01", "2024-01-03", freq="15min", inclusive="left")

commitment_upwards_deviation_price = pd.Series([100]*len(dt), index=dt) # Consumption Price
commitment_downwards_deviation_price = pd.Series([70]*len(dt), index=dt) # Production

#commitment_downwards_deviation_price[:10] = 10
#commitment_upwards_deviation_price[:10] = 10

commitment_upwards_deviation_price += -np.arange(len(dt))*0.2
commitment_downwards_deviation_price += -np.arange(len(dt))*0.2

commitment_quantities = pd.Series([0]*len(dt), index=dt)

ems_constraints = pd.DataFrame(columns=COLUMNS, index=dt)
ems_constraints["derivative max"] = 30
ems_constraints["derivative min"] = -30

n_devices = 1

soc_at_start = 5

soc_min = 10
soc_max = 90

storage_efficiency = 1
roundtrip_efficiency = .9

# Base device constraints
dc_main =  pd.DataFrame(columns=COLUMNS, index=dt)
dc_main["min"] = (soc_min - soc_at_start) * timedelta(hours=1) / resolution
dc_main["max"] =  (soc_max - soc_at_start) * timedelta(hours=1) / resolution

dc_main["efficiency"] = storage_efficiency

dc_main["derivative max"] = 5
dc_main["derivative min"] = -5

dc_main["derivative down efficiency"] = roundtrip_efficiency ** .5
dc_main["derivative up efficiency"] = roundtrip_efficiency ** .5

d = 0

#dc_main["max"][-20] = (110 - soc_at_start) * timedelta(hours=1) / resolution
dc_main["min"][30+d:40+d] = (60 - soc_at_start) * timedelta(hours=1) / resolution
dc_main["max"][50+d:60+d] = (40 - soc_at_start) * timedelta(hours=1) / resolution
dc_main["min"][50+d:60] = (40 - soc_at_start) * timedelta(hours=1) / resolution
dc_main["min"][70+d:80+d] = (60 - soc_at_start) * timedelta(hours=1) / resolution
dc_main["max"][90+d:100+d] = (20 - soc_at_start) * timedelta(hours=1) / resolution

# dc_main["equals"][-20] = (110 - soc_at_start) * timedelta(hours=1) / resolution

device_constraints = []

#device_constraints.append(dc_main)

for i in range(n_devices):
    dc = dc_main.copy()
    #dc["derivative down efficiency"] += 1
    device_constraints.append(dc)

# # Add some inflexible
# dc =  pd.DataFrame(columns=COLUMNS, index=dt)
# dc["derivative equals"] = 1
# dc["derivative equals"][10:20] = -2
# dc["derivative equals"][20:40] = -1.5
# dc["derivative equals"][40:50] = -2
# dc["derivative equals"][70:80] = -2
# dc *= -1
# device_constraints.append(dc)


In [ ]:
planned_power_per_device, planned_costs, results, model = device_scheduler(
    device_constraints,
    ems_constraints,
    [commitment_quantities],
    [commitment_downwards_deviation_price],
    [commitment_upwards_deviation_price],
    initial_stock = soc_at_start * (timedelta(hours=1) / resolution),
    ems_flow_relaxed = False,
    device_stock_relaxed = True,
    ems_flow_relaxation_cost = 20000,
    stock_relaxation_cost = 20000,
)

In [ ]:
from flexmeasures.utils.calculations import integrate_time_series
soc_schedule = integrate_time_series(
        planned_power_per_device[0],
        soc_at_start,
        up_efficiency=roundtrip_efficiency**0.5,
        down_efficiency=roundtrip_efficiency**0.5,
        storage_efficiency=storage_efficiency,
        decimal_precision=6,
    )

In [ ]:
def get_soc(d):
    d = d  * 2
    import numpy as np
    

    resolution = timedelta(minutes=15)
    dt = pd.date_range("2024-01-01", "2024-01-03", freq="15min", inclusive="left")
    
    commitment_upwards_deviation_price = pd.Series([100]*len(dt), index=dt) # Consumption Price
    commitment_downwards_deviation_price = pd.Series([70]*len(dt), index=dt) # Production
    
    #commitment_downwards_deviation_price[:10] = 10
    #commitment_upwards_deviation_price[:10] = 10
    
    commitment_upwards_deviation_price += -np.arange(len(dt))*0.2
    commitment_downwards_deviation_price += -np.arange(len(dt))*0.2
    
    commitment_quantities = pd.Series([0]*len(dt), index=dt)
    
    ems_constraints = pd.DataFrame(columns=COLUMNS, index=dt)
    ems_constraints["derivative max"] = 30
    ems_constraints["derivative min"] = -30
    
    n_devices = 1
    
    soc_at_start = 5
    
    soc_min = 10
    soc_max = 90
    
    storage_efficiency = 1
    roundtrip_efficiency = .9
    
    # Base device constraints
    dc_main =  pd.DataFrame(columns=COLUMNS, index=dt)
    dc_main["min"] = (soc_min - soc_at_start) * timedelta(hours=1) / resolution
    dc_main["max"] =  (soc_max - soc_at_start) * timedelta(hours=1) / resolution
    
    dc_main["efficiency"] = storage_efficiency
    
    dc_main["derivative max"] = 5
    dc_main["derivative min"] = -5
    
    dc_main["derivative down efficiency"] = roundtrip_efficiency ** .5
    dc_main["derivative up efficiency"] = roundtrip_efficiency ** .5
    
    
    #dc_main["max"][-20] = (110 - soc_at_start) * timedelta(hours=1) / resolution
    dc_main["min"][30+d:40+d] = (60 - soc_at_start) * timedelta(hours=1) / resolution
    dc_main["max"][50+d:60+d] = (40 - soc_at_start) * timedelta(hours=1) / resolution
    dc_main["min"][50+d:60+d] = (40 - soc_at_start) * timedelta(hours=1) / resolution
    dc_main["min"][70+d:80+d] = (60 - soc_at_start) * timedelta(hours=1) / resolution
    dc_main["max"][90+d:100+d] = (20 - soc_at_start) * timedelta(hours=1) / resolution
    
    # dc_main["equals"][-20] = (110 - soc_at_start) * timedelta(hours=1) / resolution
    
    device_constraints = []
    
    #device_constraints.append(dc_main)
    
    for i in range(n_devices):
        dc = dc_main.copy()
        #dc["derivative down efficiency"] += 1
        device_constraints.append(dc)

    planned_power_per_device, planned_costs, results, model = device_scheduler(
        device_constraints,
        ems_constraints,
        [commitment_quantities],
        [commitment_downwards_deviation_price],
        [commitment_upwards_deviation_price],
        initial_stock = soc_at_start * (timedelta(hours=1) / resolution),
        ems_flow_relaxed = False,
        device_stock_relaxed = True,
        ems_flow_relaxation_cost = 20000,
        stock_relaxation_cost = 20000,
    )

    from flexmeasures.utils.calculations import integrate_time_series
    soc_schedule = integrate_time_series(
        planned_power_per_device[0],
        soc_at_start,
        up_efficiency=roundtrip_efficiency**0.5,
        down_efficiency=roundtrip_efficiency**0.5,
        storage_efficiency=storage_efficiency,
        decimal_precision=6,
    )


    return dc_main, soc_schedule

    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.animation as animation
import warnings
import pandas as pd
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

fig, ax = plt.subplots(figsize=(20,7))

def update_plot(d):
    ax.cla()
    dc_main, soc_schedule = get_soc(2*d)

    ax.plot(dc_main[["max", "min"]] * resolution / timedelta(hours=1) + soc_at_start, label=["max", "min"], ls="dotted", lw=3)

    ax.fill_between(dc_main.index, dc_main["min"] * resolution/timedelta(hours=1) + soc_at_start,
                     dc_main["max"] * resolution / timedelta(hours=1) + soc_at_start, alpha= 0.1)
    ax.plot(soc_schedule, label="State Of Charge")
    ax.grid()
    plt.ylim(-1, 101)
    ax.axhline(90)
    ax.axhline(10)
    ax.plot(soc_schedule, label="State Of Charge")
    ax.xaxis.set_major_locator(md.HourLocator(interval=2))
    ax.xaxis.set_minor_locator(md.HourLocator(interval=1))
    ax.xaxis.grid(True, which="minor")
    ax.xaxis.set_major_formatter(md.DateFormatter("%H:%M"))
    ax.set(xlim=[dc_main.index[0], dc_main.index[-1]], ylim=[-1,101])    
    plt.rc("font", **{"size" : 12})
    plt.tick_params(axis="x", which="minor")
    ax.legend()
update_plot(0)


ani = animation.FuncAnimation(fig=fig, func=update_plot, frames=20, interval=1000)
ani.save("prueba.mp4")

In [ ]:
from IPython.display import HTML
HTML(ani.to_html5_video())


In [ ]:
ani.save("prueba.mp4")

In [ ]:
results.solver.termination_condition
pd.Series(model.device_stock_slack_lower.extract_values()).plot()
plt.show()
pd.Series(model.device_stock_slack_upper.extract_values()).plot()
results.solver.termination_condition

In [ ]:
planned_costs

In [ ]:
import matplotlib.pyplot as plt
for i, power in enumerate(planned_power_per_device):
    plt.plot(power, label=i)
plt.legend()
plt.grid()
plt.tight_layout()

In [ ]:
min(sum(planned_power_per_device)), max(sum(planned_power_per_device))

In [ ]:
plt.plot(sum(planned_power_per_device))

In [ ]:
plt.plot(commitment_upwards_deviation_price, label="up")
plt.plot(commitment_downwards_deviation_price, label="down")

plt.legend()